## A Battle of Neighbors

This is the capstone project for IMB Data Science Professional Certificate for Coursera.

### Intoduction
New York City (NYC), Toronto, Tokyo -- three major international megacities. <br/>
<br/>
The first two are large-multicultural cities where English — the language of business — is widely spoken. One the other hand, Japanese the the business language in Tokyo with growing number of people communicating in English. Cities like NYC and Toronto Cities are mostly known for their boroughs and the cultures of individual neighborhoods, whereas Tokyo is renown for being an in-gathering of Japanese citizens from all over the country. Nevertheless, all three of them has huge potential for anyone move in if they have the education and the work ethic necessary to be successful. <br/>
<br/>
New York is famous for the boroughs of Manhattan, Queens, Brooklyn, Staten Island, and the nearby cities of Long Island. It is very big, sometime when people refers to NYC they just mean Manhattan. Within these boroughs, NYC can be divided into neighborhoods like Chinatown and Wallstreet, which are very distinct in terms of life style and culture. With almost 200 languages spoken, 40 percent of the NYC population was born outside of the United States. NYC is truely a ever-changing rainbow of worlds. <br/>
<br/>
Toronto, another example of multiculture metropolitan, has over 250 ethnicities and 170 languages. Unlike NYC though, the neighborhoods are less segregated by culture. They still have neighborhoods like chinatown and kinston market, but the neighborhoods are more represented by life styles, at least I think so. <br/>
<br/>
Tokyo is divided up into various districts called ‘wards’. It is more known for its cultural landmarks than its diversity. This is because the city population is still around 90% Japanese, and many businesspeople and tourists who have lived there are only temporary residents. It is highly likely that people will be shockedby the stoic silence on the trains, and the differences in cultural etiquette when first come to Tokyo. However, with growing business and uniquenss of it, Tokyo now attract more and more global citizens. 
<br/>


### Business Problem
Since these three cities have some similarities and some striking differences, one interesting thing to compare is that:
- How are their neighborhoods clustered. How similar/disimilar are they in terms of neighborhood categories? Is toronto much more similar in in terms of their neighoods venues with NYC compared to Tokyo?
Another interesting question we can answer is that:
- Since they all have a lot of potential, if we want to open a new business in one of them, which type of venue is more likely to succed. Or if we have a given venue to start, which neighborhood of them is more likely to accept it?

In [1]:
# Import packages
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

### Data

In order to do this project, we scriped Toronto, NYC, and tokyo's boroughs/wards from wikipedia:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M <br/>
https://en.wikipedia.org/wiki/Special_wards_of_Tokyo)<br/>
*Note: Only use manhattan data since NYC is very big and people often refer Manhattan as the city* <br/>
We then obtained the geolocation of each postal_codes and wards using geocoder. Next, we request API form **foursquare** to access the top 100 venues (if exist) within radius of 1000 meters for each neighboorhood.

1. The NYC data:

In [4]:
df_NYC = pd.read_csv('Manhattan_grouped.csv').drop('Unnamed: 0', axis=1)
df_NYC.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.00,...,0.0,0.00,0.01,0.0,0.0,0.00,0.02,0.0,0.01,0.00
1,Carnegie Hill,0.0,0.0,0.0,0.00,0.02,0.00,0.0,0.0,0.01,...,0.0,0.01,0.00,0.0,0.0,0.01,0.03,0.0,0.01,0.04
2,Central Harlem,0.0,0.0,0.0,0.04,0.02,0.00,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.0,0.00,0.02,0.0,0.00,0.02
3,Chelsea,0.0,0.0,0.0,0.00,0.03,0.00,0.0,0.0,0.00,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.03
4,Chinatown,0.0,0.0,0.0,0.00,0.03,0.00,0.0,0.0,0.00,...,0.0,0.01,0.00,0.0,0.0,0.02,0.00,0.0,0.00,0.00


In [5]:
df_NYC.shape

(40, 319)

This dataset contains the neighoorhood information (column: Neighborhood), and within each neighborhood, the frequency of one type of venue (the rest of the column). The higher the number is, the more venues in this neighborhood. 

2. The Toronto data:

In [8]:
df_toronto = pd.read_csv('toronto_grouped.csv').drop('Unnamed: 0', axis=1)
df_toronto.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M1B,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,M1C,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,M1E,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
3,M1G,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,M1H,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333,0.0


This dataset contains the neighoorhood information (column: Postal code), and within each postal code regin, the frequency of one type of venue (the rest of the column). The higher the number is, the more venues in this neighborhood. 

3. The Tokyo data:

In [11]:
df_tokyo = pd.read_csv('tokyo_grouped.csv').drop('Unnamed: 0', axis=1)
df_tokyo.head()

,Borough,ATM,Accessories Store,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Wine Bar,Wine Shop,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Akabane,0.01,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.02,0.0,0.00
1,Arakawa,0.00,0.0,0.0,0.0,0.0,0.010417,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00
2,Bunkyō,0.00,0.0,0.0,0.0,0.0,0.010000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.04,0.0,0.01
3,Chiyoda,0.00,0.0,0.0,0.0,0.0,0.010000,0.02,0.0,0.0,...,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.03,0.0,0.01
4,Chūō,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.01,0.0,0.01


This dataset contains the neighoorhood information (column: Borough), and within each postal code regin, the frequency of one type of venue (the rest of the column). The higher the number is, the more venues in this neighborhood.

Next, we will use these three datasets to perform a KMeans clustering study to identify neighborhoods clusters. Then we will compare the clusters within each city to derive a segementation report. Next we would like to compare the similarity and disimilarity of the cities using distance matrix.